Le but de ce code est de modifier comme il se doit le data set ytrain pour avoir des données par heures et dans un carré


In [29]:
import folium
import pandas as pd
from folium.plugins import TimestampedGeoJson
from shapely.geometry import Point
import numpy as np
from math import radians, sin, cos, sqrt, atan2
from branca.colormap import linear

In [21]:
data = pd.read_csv("hourly_df_semaine2229.csv")
df_real = pd.DataFrame(data)
df_real['taux_dispo'] =  (df_real['Available'] + df_real['Offline']/3)/3
df_real = df_real[['timestamp','latitude','longitude','taux_occup','taux_dispo']]


df_real.head(20)


,timestamp,latitude,longitude,taux_occup,taux_dispo
0,2020-01-22 01:00:00+00:00,48.855667,2.354089,0.666667,0.333333
1,2020-01-22 01:00:00+00:00,48.864240,2.397724,0.666667,0.333333
2,2020-01-22 01:00:00+00:00,48.861016,2.283080,0.666667,0.333333
3,2020-01-22 01:00:00+00:00,48.824388,2.336857,0.666667,0.333333
4,2020-01-22 01:00:00+00:00,48.846295,2.374877,0.666667,0.333333
5,2020-01-22 01:00:00+00:00,48.889731,2.338910,0.500000,0.500000
6,2020-01-22 01:00:00+00:00,48.845265,2.411238,0.000000,1.000000
7,2020-01-22 01:00:00+00:00,48.846761,2.268339,0.333333,0.666667
8,2020-01-22 01:00:00+00:00,48.877409,2.392178,0.000000,1.000000
9,2020-01-22 01:00:00+00:00,48.878897,2.355064,0.000000,1.000000


In [22]:


# Définir les limites de la grille (basé sur Paris)
lat_min, lat_max = 48.81, 48.92
lon_min, lon_max = 2.255, 2.42
step = 0.005  # Taille d'un carré (grille de 0.005°)

In [23]:

# Fonction pour créer la grille
def creer_grille(lat_min, lat_max, lon_min, lon_max, step):
    grille = []
    latitudes = np.arange(lat_min, lat_max, step)
    longitudes = np.arange(lon_min, lon_max, step)
    
    for i in range(len(latitudes) - 1):
        for j in range(len(longitudes) - 1):
            carre = {
                "id": f"{i}_{j}",
                "lat_min": latitudes[i],
                "lat_max": latitudes[i + 1],
                "lon_min": longitudes[j],
                "lon_max": longitudes[j + 1],
                "lati": latitudes[i] + step / 2,  # Centre du carré
                "long": longitudes[j] + step / 2  # Centre du carré
            }
            grille.append(carre)
    return grille

# Fonction pour trouver le carré correspondant à un point
def trouver_carre(lat, lon, grille):
    if pd.isnull(lat) or pd.isnull(lon):  # Vérifier si lat ou lon est NaN
        return None, None, None
    for carre in grille:
        if carre["lat_min"] <= lat < carre["lat_max"] and carre["lon_min"] <= lon < carre["lon_max"]:
            return carre["id"], carre["lati"], carre["long"]
    return None, None, None

# Fonction pour associer les carrés à chaque point GPS
def associer_carres(df, grille):
    df[['carre_id', 'lati', 'long']] = df.apply(
        lambda row: pd.Series(trouver_carre(row['latitude'], row['longitude'], grille)), axis=1
    )
    return df

In [24]:


# Créer la grille
grille = creer_grille(lat_min, lat_max, lon_min, lon_max, step)



# Extraire les identifiants uniques pour correspondre les datasets
df_real['timestamp'] = pd.to_datetime(df_real['timestamp'])  # Convertir en datetime



# Ajouter les informations de la grille au dataset final
df_real = associer_carres(df_real, grille)

# Supprimer les lignes avec des valeurs manquantes critiques
df_real = df_real.dropna(subset=['latitude', 'longitude', 'carre_id'])

# Afficher un aperçu du DataFrame final
print(df_real.head(2))



                  timestamp   latitude  longitude  taux_occup  taux_dispo  \
0 2020-01-22 01:00:00+00:00  48.855667   2.354089    0.666667    0.333333   
1 2020-01-22 01:00:00+00:00  48.864240   2.397724    0.666667    0.333333   

  carre_id     lati    long  
0     9_19  48.8575  2.3525  
1    10_28  48.8625  2.3975  


In [25]:
# Fonction haversine
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Rayon de la Terre en kilomètres
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat / 2) ** 2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

In [ ]:
# Fonction pour trouver la station la plus proche
def closest_station_distance(row, df_stations):
    """
    Calcule la distance minimale entre une ligne donnée (avec 'lati' et 'long') et un ensemble de stations.
    :param row: Ligne individuelle du DataFrame df_real.
    :param df_stations: DataFrame contenant les stations avec colonnes 'latitude' et 'longitude'.
    :return: Distance minimale à la station la plus proche.
    """
    lat = row['lati']
    lon = row['long']
    df_station_dispo = df_stations[df_stations['taux_dispo'] >= 1/3]
    # Calculer toutes les distances
    df_stations['distance'] = df_stations.apply(
        lambda station: haversine(lat, lon, station['latitude'], station['longitude']),
        axis=1
    )
    # Retourner la distance minimale
    return df_stations['distance'].min(), df_stations['distance'].idxmin()

In [ ]:
# Regrouper les données par carré et t_1h, puis calculer la moyenne de k
resultats_groupes = df_real.groupby(['timestamp','carre_id']).agg(
    moyenne_occup=('taux_occup', 'mean'),
    count_occup=('taux_occup', 'count'),
    lati= ('lati', 'mean'),
    long= ('long', 'mean')
).reset_index()
resultats_groupes[['distance'] = resultats_groupes.apply(
    lambda row: closest_station_distance(
        row,
        df_real[df_real['timestamp'] == row['timestamp']]  # Filtrer les stations par instant t_1h
    ),
    axis=1
)
print(resultats_groupes['moyenne_occup'].max())
resultats_groupes.head(10)
print(resultats_groupes['moyenne_occup'].describe())

1.0
count    10920.000000
mean         0.662305
std          0.269786
min          0.000000
25%          0.416667
50%          0.666667
75%          1.000000
max          1.000000
Name: moyenne_occup, dtype: float64


In [ ]:
print(resultats_groupes['distance'].describe())


count    10920.000000
mean         0.184965
std          0.072464
min          0.025630
25%          0.136929
50%          0.191621
75%          0.237132
max          0.312576
Name: distance, dtype: float64


In [ ]:
# Sauvegarder le DataFrame final dans un nouveau fichier CSV
resultats_groupes.to_csv("station_agg.csv", index=False)


Table stat plus proche